In [30]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

In [31]:
# Load the trained model and the scaler
model = load_model("trained_model.h5")
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename)

In [32]:
def predict_func(data):
    # Preprocess the data
    new_data = data.interpolate(method='cubic', limit_direction='both')
    new_data = new_data.filter(['Close'])
    scaled_data = scaler.transform(new_data.values.reshape(-1, 1))

    # Prepare the input for prediction
    x_test = np.reshape(scaled_data, (1, scaled_data.shape[0], 1))

    # Make predictions
    predictions = model.predict(x_test)
    
    # Inverse transform the predictions
    predicted_data = scaler.inverse_transform(predictions)

    return predicted_data.flatten().tolist()

In [33]:
pdf = pd.read_csv('sample_input.csv')

In [34]:
predict_func(pdf)

1/1 [==============================] - 1s 653ms/step


[7218.70654296875, 7245.041015625]

In [35]:
def evaluate():
    # Input the csv file
    """
    Sample evaluation function
    Don't modify this function
    """
    df = pd.read_csv('sample_input.csv')
     
    actual_close = np.loadtxt('sample_close.txt')
    
    pred_close = predict_func(df)
    
    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close-pred_close))


    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close
    
    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)-np.array(pred_prev)
    actual_dir = np.array(actual_curr)-np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir*actual_dir)>0)*100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')

In [36]:
if __name__== "__main__":
    evaluate()

1/1 [==============================] - 0s 29ms/step
Mean Square Error: 630.892529
Directional Accuracy: 100.0
